In [15]:
import zipfile
import csv
import requests
import cv2
import matplotlib.pyplot as plt
import glob 
import random
import os


def create_dict_from_csv(labels_csv_path):
    dict_from_csv = {}
    with open(labels_csv_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        rows = list(reader)
        for row in rows:
            dict_from_csv[row[0]] = int(row[1])
    return dict_from_csv

def get_rows_by_filename(csv_path, file_name):
    matching_rows = []
    with open(csv_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        for row in reader:
            if row[5] == file_name:
                matching_rows.append(row)
    return matching_rows

def convert(size, box, label):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (label, x,y,w,h)

def annots_in_yolo(image_path, csv_paths, labels, update_csv=False, write_txt=False):
    file_name = os.path.basename(image_path)
    img_dir = os.path.dirname(image_path)
    txt_file = os.path.join(img_dir, os.path.splitext(file_name)[0] + '.txt')
    annots = []
    for csv_file in csv_paths:
        rows = get_rows_by_filename(csv_file, file_name)
        for row in rows:
            label, x1, y1, bbox_w, bbox_h, filename, img_width, img_height = row
            if label is None or label == '':
                continue
                
            label_idx = labels[label]
            x2, y2 = int(x1) + int(bbox_w), int(y1) + int(bbox_h)
            sizes = (int(img_width), int(img_height))
            boxes = int(x1), int(x2), int(y1), int(y2)
            yolo_format = convert(sizes, boxes, label_idx)
            if update_csv:
                new_row = [label, x1, y1, bbox_w, bbox_h, filename, img_width, img_height, yolo_format]
                directory = os.path.dirname(csv_file)
                filename = os.path.splitext(os.path.basename(csv_file))[0]
                
                new_csv_file_path = os.path.join(directory, f"{filename}-yolo.csv")
                with open(new_csv_file_path, "a", newline="") as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow(new_row)
            
            annots.append(yolo_format)
    print(file_name, annots)
    if write_txt:
        with open(txt_file, "w") as txt_file:
            for annot in annots:
                txt_file.write(f"{annot[0]} {annot[1]} {annot[2]} {annot[3]} {annot[4]}\n")
                print(f"{annot[0]} {annot[1]} {annot[2]} {annot[3]} {annot[4]}")
    return annots

def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

def plot_box(image, bboxes, labels, label_dict):
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))
                
        class_name = next((k for k, v in label_dict.items() if v == labels[box_num]), None)
        cv2.rectangle(
            image, 
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
        cv2.putText(
            image,
            class_name,
            (xmin, ymin+20),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_AA
        )
    return image

def plot(image_paths, csv_path, labels_csv_path, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.jpeg'))
    
    all_images.sort()

    num_images = len(all_images)
    
    label_dict = create_dict_from_csv(labels_csv_path)
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        print(image_name)
        image = cv2.imread(all_images[j])
        label_lines = annots_in_yolo(image_name, csv_path, label_dict)
        bboxes = []
        labels = []
        for label_line in label_lines:
            label = label_line[0]
            x_c, y_c, w, h = label_line[1:]
            x_c = float(x_c)
            y_c = float(y_c)
            w = float(w)
            h = float(h)
            bboxes.append([x_c, y_c, w, h])
            labels.append(label)
        result_image = plot_box(image, bboxes, labels, label_dict)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

# plot(
#     image_paths='../vegnet_images/', 
#     csv_path='../vegnet_annotations/vegnet-annots-all-updated.csv',
#     labels_csv_path='../vegnet_annotations/labels.csv',
#     num_samples=4,
# )

In [16]:
def save_annots(
    image_paths='../vegnet_images', 
    csv_path_train='../vegnet_annotations/vegnet-annots-train-updated-counted.csv',
    csv_path_val='../vegnet_annotations/vegnet-annots-val-updated-counted.csv',
    labels_csv_path='../vegnet_annotations/labels.csv',
):
    
    all_images = []
    all_images.extend(glob.glob(image_paths+'/train/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/train/*.jpeg'))
    all_images.extend(glob.glob(image_paths+'/val/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/val/*.jpeg'))
    
    all_images.sort()

    num_images = len(all_images)

    label_dict = create_dict_from_csv(labels_csv_path)

    for i in range(num_images):
        image_name = all_images[i]
        label_lines = annots_in_yolo(image_name, [csv_path_train, csv_path_val], label_dict, update_csv=True, write_txt=True)
  
save_annots()

100_Bacterial Spot. (49).jpg [(2, 0.5723860589812332, 0.4623915139826423, 0.8552278820375335, 0.9016393442622951)]
2 0.5723860589812332 0.4623915139826423 0.8552278820375335 0.9016393442622951
101_Bacterial Spot. (5).jpg [(2, 0.43024830699774264, 0.4573643410852713, 0.455079006772009, 0.3178294573643411)]
2 0.43024830699774264 0.4573643410852713 0.455079006772009 0.3178294573643411
102_Bacterial Spot. (50).jpg [(2, 0.5048283261802575, 0.4643211100099108, 0.990343347639485, 0.908820614469772)]
2 0.5048283261802575 0.4643211100099108 0.990343347639485 0.908820614469772
103_Bacterial Spot. (51).jpg [(2, 0.5299506694855532, 0.5160031225604996, 0.7906976744186047, 0.7056986729117877)]
2 0.5299506694855532 0.5160031225604996 0.7906976744186047 0.7056986729117877
104_Bacterial Spot. (52).jpg [(2, 0.08915486077186126, 0.6871406168322007, 0.07962872496336101, 0.1374803972817564), (2, 0.3646800195407914, 0.46027182435964453, 0.28969223253541765, 0.20752744380554103), (2, 0.7024914509037616, 0.58